In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
import xgboost as xgb

In [2]:
train = pd.read_csv('data/train.csv')
macro = pd.read_csv('data/macro.csv')

In [3]:
print(train.shape)
train.head()

(30471, 292)


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [4]:
print(macro.shape)
macro.head()

(2485, 100)


,timestamp,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_deflator,balance_trade,balance_trade_growth,usdrub,...,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,2010-01-01,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
1,2010-01-02,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
2,2010-01-03,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
3,2010-01-04,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.905,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
4,2010-01-05,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.836,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN


Учитывая, что имеются данные по макроэкономике этой страны, то логично, что надо бы их использовать. Объединим таблицы train и macro, и "будем радоваться", что теперь имеем 390 признаков

In [5]:
both_table = train.join(macro.set_index('timestamp'), on='timestamp')

In [6]:
print(both_table.shape)
both_table.head()

(30471, 391)


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0


Заменим значения в некоторых столбцах, на более восприимчевые

In [7]:
both_table = both_table.fillna(-1)
both_table = both_table.replace('yes', 1)
both_table = both_table.replace('no', 0)

In [8]:
both_table.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,1,2011-08-20,43,27.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
1,2,2011-08-23,34,19.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
2,3,2011-08-27,43,29.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
3,4,2011-09-01,89,50.0,9.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0
4,5,2011-09-05,77,77.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,271.0,6943.0,565.0,0.45356,1240.0,269768.0,22.37,64.12,23587.0,230310.0


Ну и со временем

In [9]:
dateTime = pd.to_datetime(both_table["timestamp"])
both_table["month"] = dateTime.map(lambda x: x.month)
both_table = both_table.drop(['timestamp', 'id'], axis=1)

Построим матрицу коррелирующих признаков

Не смог(

In [10]:
corr = both_table.corr()
column_table = len(corr.columns)
f = plt.figure()
fig, ax = plt.subplots(figsize=(column_table, column_table))
ax.matshow(corr)
plt.xticks(range(len(corr.columns)), corr.columns)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.show()
f.savefig("corr.pdf", bbox_inches='tight')

<Figure size 432x288 with 0 Axes>

MemoryError: 

<Figure size 27648x27648 with 1 Axes>

Тогда ограничимся таблицей train

In [10]:
X = train.fillna(-1)
X = X.replace('yes', 1)
X = X.replace('no', 0)
dateTime = pd.to_datetime(X["timestamp"])
X["month"] = dateTime.map(lambda x: x.month)
X = X.drop(['timestamp', 'id'], axis=1)
Y = train['price_doc']

In [34]:
def replaceWithCategorial(table, key):
    values = table[key].unique().tolist()
    table[key] =  table[key].map(lambda x: values.index(x))
    return table

In [35]:
X = replaceWithCategorial(X, "product_type")
X = replaceWithCategorial(X, "sub_area")
X = replaceWithCategorial(X, "ecology")

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Воспользуемся дефолтным XGBRegressor

In [37]:
xgbRegressor = xgb.XGBRegressor()
xgbRegressor.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [40]:
pred = xgbRegressor.predict(X_test)
print(metrics.mean_squared_error(Y_test, pred))

3906228285.0458007


Поиграем с параметрами:

In [41]:
gridSearch = model_selection.GridSearchCV(xgbRegressor,
             {'max_depth': [5, 7, 9], 'colsample_bytree': [0.5, 0.7, 0.9],
              'n_estimators': [50, 100, 300]}, 
              scoring='neg_mean_squared_error')
gridSearch.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [50, 100, 300], 'colsample_bytree': [0.5, 0.7, 0.9], 'max_depth': [5, 7, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [45]:
print(gridSearch.best_params_)

{'n_estimators': 300, 'colsample_bytree': 0.9, 'max_depth': 9}


Применяем и тестируем

In [48]:
xgbRegressor.set_params(**gridSearch.best_params_)
xgbRegressor.fit(X_train, Y_train)
pred = xgbRegressor.predict(X_test)

In [49]:
print(metrics.mean_squared_error(Y_test, pred))

1047008821.1507775
